Task 1

In [136]:
import requests
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from collections import Counter
from datetime import datetime
from urllib.parse import unquote

WIKIDATA_API_ENDPOINT = "https://www.wikidata.org/w/api.php"
WIKIPEDIA_API_ENDPOINT = "https://en.wikipedia.org/w/api.php"

HEADERS = {"User-Agent": "uni_coursework/2.0; nat.1.roongjirarat@kcl.ac.uk"}

PARAMS_QUERY_SEARCH = {
    "action":"query",
    "format":"json",
    "formatversion":"latest",
    "list":"search",
    "srsearch": "haswbstatement:P166=Q185667",
    "srlimit":"max"
}

PARAMS_GETCONTENT={
    "action": "query",
    "format": "json",
    "titles": "",
    "prop": "extracts",
    "explaintext": True,
}

PARAMS_WBGETENTITIES_LABELS = {
    "action": "wbgetentities",
    "format": "json",
    "ids": "",
    "sites": "",
    "props": "labels",
    "languages": "en",
    "sitefilter": "enwiki",
    "utf8": 1,
    "ascii": 1,
    "formatversion": "latest"
}

PARAMS_WBGETENTITIES_SITES = {
    "action": "wbgetentities",
    "format": "json",
    "ids": "",
    "sites": "",
    "props": "sitelinks/urls",
    "languages": "en",
    "sitefilter": "enwiki",
    "utf8": 1,
    "ascii": 1,
    "formatversion": "latest"
}

PARAMS_WBGETENTITIES_CLAIMS = {
    "action": "wbgetentities",
    "format": "json",
    "ids": "",
    "props": "claims",
    "languages": "en",
    "sitefilter": "",
    "formatversion": "latest"
}

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nat_rng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nat_rng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def get_turing_award_recipients():
    acm_award_entities = []
    search_response = requests.get(WIKIDATA_API_ENDPOINT, headers=HEADERS, params=PARAMS_QUERY_SEARCH)
    data = search_response.json()
    for result in data['query']['search']:
        acm_award_entities.append(result['title'])
    return acm_award_entities

print(get_turing_award_recipients())

Task 2

In [154]:
def get_wikipedia_content(entity_id):
    PARAMS_WBGETENTITIES_SITES["ids"] = entity_id
    wbgetentities_response = requests.get(WIKIDATA_API_ENDPOINT, headers=HEADERS, params=PARAMS_WBGETENTITIES_SITES)
    wbgetentities_data = wbgetentities_response.json()
    recipient_name = wbgetentities_data["entities"][entity_id]["sitelinks"]["enwiki"]["url"].split("https://en.wikipedia.org/wiki/")[1]

    PARAMS_GETCONTENT["titles"] = unquote(recipient_name)
    wbgetentities_response = requests.get(WIKIPEDIA_API_ENDPOINT, headers=HEADERS, params=PARAMS_GETCONTENT)
    wbgetentities_data = wbgetentities_response.json()
    content = next(iter(wbgetentities_data["query"]["pages"].values()))["extract"]
    return content

print(get_wikipedia_content("Q80"))

{'pageid': 30034, 'ns': 0, 'title': 'Tim Berners-Lee', 'extract': 'Sir Timothy John Berners-Lee  (born 8 June 1955), also known as TimBL, is an English computer scientist best known as the inventor of the World Wide Web. He is a Professorial Fellow of Computer Science at the University of Oxford and a professor at the Massachusetts Institute of Technology (MIT). Berners-Lee proposed an information management system on 12 March 1989, then implemented the first successful communication between a Hypertext Transfer Protocol (HTTP) client and server via the Internet in mid-November.Berners-Lee is the director of the World Wide Web Consortium (W3C), which oversees the continued development of the Web. He co-founded (with his then wife-to-be Rosemary Leith) the World Wide Web Foundation. He is a senior researcher and holder of the 3Com founder\'s chair at the MIT Computer Science and Artificial Intelligence Laboratory (CSAIL). He is a director of the Web Science Research Initiative (WSRI) an

Task 3

In [ ]:
wiki_data_dict = {"gender": "P21", "birth_date": "P569", "birth_city": "P19", 
                  "birth_country": "P17", "employer": "P108", "educated_at": "P69"}

def get_wikidata_label(entity_id):
    PARAMS_WBGETENTITIES_LABELS["ids"] = entity_id
    wbgetentities_response = requests.get(WIKIDATA_API_ENDPOINT, headers=HEADERS, params=PARAMS_WBGETENTITIES_LABELS)
    wbgetentities_data = wbgetentities_response.json()
    labels = next(iter(wbgetentities_data["entities"].values()))["labels"]
    value = labels["en"]["value"]
    return value

def get_wikidata_claims(entity_id):
    PARAMS_WBGETENTITIES_CLAIMS["ids"] = entity_id
    wbgetentities_response = requests.get(WIKIDATA_API_ENDPOINT, headers=HEADERS, params=PARAMS_WBGETENTITIES_CLAIMS)
    wbgetentities_data = wbgetentities_response.json()
    claims = next(iter(wbgetentities_data["entities"].values()))["claims"]
    return claims

def get_dict_values(entity_id):
    claims = get_wikidata_claims(entity_id)
    try:
        name = get_wikidata_label(entity_id).split(" (")[0]
    except KeyError:
        name = None
    try:
        intro = get_wikipedia_content(entity_id).split("\n\n\n")[0]
    except KeyError:
        intro = None   
    try: 
        gender = get_wikidata_label(claims[wiki_data_dict["gender"]][0]["mainsnak"]["datavalue"]["value"]["id"])
    except KeyError:
        gender = None
    try:
        birth_date = datetime.strptime(claims[wiki_data_dict["birth_date"]][0]["mainsnak"]["datavalue"]["value"]["time"], "+%Y-%m-%dT%XZ").strftime("%d %B %Y")
    except ValueError:
        birth_date = datetime.strptime(claims[wiki_data_dict["birth_date"]][0]["mainsnak"]["datavalue"]["value"]["time"], "+%Y-00-00T%XZ").strftime("%Y")
    except KeyError:
        birth_date = None
    try:
        birth_city = get_wikidata_label(claims[wiki_data_dict["birth_city"]][0]["mainsnak"]["datavalue"]["value"]["id"])
        birth_city_claims = get_wikidata_claims(claims[wiki_data_dict["birth_city"]][0]["mainsnak"]["datavalue"]["value"]["id"])
        birth_country = get_wikidata_label(birth_city_claims[wiki_data_dict["birth_country"]][0]["mainsnak"]["datavalue"]["value"]["id"])
        birth_place = "{}, {}".format(birth_city, birth_country)
    except KeyError:
        birth_place = None
    try:
        employer_list = []
        if len(claims[wiki_data_dict["employer"]]) == 1:
            employer = get_wikidata_label(claims[wiki_data_dict["employer"]][0]["mainsnak"]["datavalue"]["value"]["id"])
        else:
            for i in range(len(claims[wiki_data_dict["employer"]])):
                employer = get_wikidata_label(claims[wiki_data_dict["employer"]][i]["mainsnak"]["datavalue"]["value"]["id"])
                employer_list.append(employer)
            employer = employer_list
    except KeyError:
        employer = None
    try:
        education_list = []
        if len(claims[wiki_data_dict["educated_at"]]) == 1:
            education = get_wikidata_label(claims[wiki_data_dict["educated_at"]][0]["mainsnak"]["datavalue"]["value"]["id"])
        else:
            for i in range(len(claims[wiki_data_dict["educated_at"]])):
                education = get_wikidata_label(claims[wiki_data_dict["educated_at"]][i]["mainsnak"]["datavalue"]["value"]["id"])
                education_list.append(education)
            education = education_list
    except KeyError:
        education = None
    return name, intro, gender, birth_date, birth_place, employer, education


award_winners = {"name": [], "intro": [], "gender": [], "birth_date": [], 
                    "birth_place": [], "employer": [], "educated_at": []}
acm_award_winners = get_turing_award_recipients()
for entity_id in acm_award_winners:
    name, intro, gender, birth_date, birth_place, employer, education = get_dict_values(entity_id)
    award_winners["name"].append(name)
    award_winners["intro"].append(intro)
    award_winners["gender"].append(gender)
    award_winners["birth_date"].append(birth_date)
    award_winners["birth_place"].append(birth_place)
    award_winners["employer"].append(employer)
    award_winners["educated_at"].append(education)

Task 4

In [ ]:
for name in sorted(award_winners["name"]):
    print(name)

Adi Shamir
Alan Kay
Alan Perlis
Alfred Aho
Allen Newell
Amir Pnueli
Andrew Yao
Barbara Liskov
Bob Kahn
Butler Lampson
Charles Bachman
Charles P. Thacker
Dana Scott
David A. Patterson
Dennis M. Ritchie
Donald Knuth
Douglas Engelbart
E. Allen Emerson
Edgar F. Codd
Edmund M. Clarke
Edsger W. Dijkstra
Edward Feigenbaum
Edwin Catmull
Fernando J. Corbató
Frances E. Allen
Fred Brooks
Geoffrey Hinton
Herbert Simon
Iosif Sifakis
Ivan Sutherland
Jack Dongarra
James H. Wilkinson
Jeffrey David Ullman
Jim Gray
John Backus
John Cocke
John Edward Hopcroft
John L. Hennessy
John McCarthy
Judea Pearl
Juris Hartmanis
Ken Thompson
Kenneth E. Iverson
Kristen Nygaard
Leonard Adleman
Leslie Lamport
Leslie Valiant
Manuel Blum
Martin Edward Hellman
Marvin Minsky
Maurice Wilkes
Michael O. Rabin
Michael Stonebraker
Niklaus Wirth
Ole-Johan Dahl
Pat Hanrahan
Peter Naur
Raj Reddy
Richard E. Stearns
Richard Hamming
Richard M. Karp
Robert Tarjan
Robert W. Floyd
Robin Milner
Ron Rivest
Shafrira Goldwasser
Silvio Mical

Task 5

In [142]:
award_winners_intro = pd.DataFrame(columns=["winner_name", "count_words", "count_sentences", "count_paragraphs", "common_words"])
award_winners_intro["winner_name"] = award_winners["name"]

def get_intro_stats(intro): 
    count_words = sum([word.strip(string.punctuation).isalnum() for word in intro.split()])
    count_sentences = len(sent_tokenize(intro))
    count_paragraphs = len(intro.split("\n"))
    common_words = Counter([word.strip(string.punctuation) for word in intro.split()]).most_common(15)
    return count_words, count_sentences, count_paragraphs, common_words

intro_stats = {"word_count": [], "sentence_count": [], "paragraph_count": [], "common_words": []}
for intro in award_winners["intro"]:
    count_word, count_sentences, count_paragraphs, common_words = get_intro_stats(intro)
    intro_stats["word_count"].append(count_word)
    intro_stats["sentence_count"].append(count_sentences)
    intro_stats["paragraph_count"].append(count_paragraphs)
    intro_stats["common_words"].append(common_words)

award_winners_intro["count_words"] = intro_stats["word_count"]
award_winners_intro["count_sentences"] = intro_stats["sentence_count"]
award_winners_intro["count_paragraphs"] = intro_stats["paragraph_count"]
award_winners_intro["common_words"] = intro_stats["common_words"]

award_winners_intro.head()

,winner_name,count_words,count_sentences,count_paragraphs,common_words
0,Tim Berners-Lee,344,14,1,"[(the, 36), (of, 21), (and, 14), (Web, 10), (H..."
1,Yoshua Bengio,90,4,2,"[(and, 6), (the, 5), (of, 4), (Bengio, 3), (fo..."
2,Geoffrey Hinton,172,6,1,"[(the, 12), (and, 8), (of, 7), (for, 5), (in, ..."
3,Donald Knuth,178,7,2,"[(the, 20), (of, 13), (and, 11), (Knuth, 5), (..."
4,Richard M. Karp,89,2,1,"[(in, 6), (and, 5), (the, 5), (of, 5), (for, 3..."


Task 5

In [152]:
def process_common_words(intro):
    word_filter = set(stopwords.words('english') + list(string.punctuation))
    tokenized_intro = word_tokenize(intro)
    common_words = Counter([word for word in tokenized_intro if word not in word_filter]).most_common(10)
    return common_words

common_words_after_preprocessing = [process_common_words(intro) for intro in award_winners["intro"]]
award_winners_intro["common_words_after_preprocessing"] = common_words_after_preprocessing

print(award_winners_intro.head())

       winner_name  count_words  count_sentences  count_paragraphs  \
0  Tim Berners-Lee          344               14                 1   
1    Yoshua Bengio           90                4                 2   
2  Geoffrey Hinton          172                6                 1   
3     Donald Knuth          178                7                 2   
4  Richard M. Karp           89                2                 1   

                                        common_words  \
0  [(the, 36), (of, 21), (and, 14), (Web, 10), (H...   
1  [(and, 6), (the, 5), (of, 4), (Bengio, 3), (fo...   
2  [(the, 12), (and, 8), (of, 7), (for, 5), (in, ...   
3  [(the, 20), (of, 13), (and, 11), (Knuth, 5), (...   
4  [(in, 6), (and, 5), (the, 5), (of, 5), (for, 3...   

                    common_words_after_preprocessing  
0  [(Web, 12), (He, 10), (World, 5), (Wide, 5), (...  
1  [(Bengio, 3), (work, 2), (deep, 2), (learning,...  
2  [(Hinton, 3), (computer, 2), (work, 2), (neura...  
3  [(Knuth, 5), (compu